In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [3]:
# logging 
import logging
import logging.config

# parse configuration files
import configparser

# parse command line arguments
import argparse

# handle file paths in a sane way
from pathlib import Path

In [4]:
# this works best as a global variable
# FIXME - best practice for specifying this?
logConfig = Path('./logging.cfg')
logging.config.fileConfig(logConfig.absolute())
# logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(name)s %(levelname)s: %(message)s')

logger = logging.getLogger(__name__)

In [5]:
def arg_parse():
    '''parse arguments for this script
    
    Arguments parsed:
        -c: configuration file
    
    Returns:
        :obj:argpars.ArgumentParser'''
    parser = argparse.ArgumentParser()
    
    # configuration file
    parser.add_argument('-c', type=str, required=False,
                        help='use the specified configuration file. Default is stored in ~/.config/myApp/config.ini')
    
    parser.add_argument('-q', type=str, required=False,
                       help='this is for testing only')
    
    args, unknown = parser.parse_known_args()
    logging.info(f'discarding unknwon commandline arguments: {unknown}')
    
    return args

In [42]:
def read_config(cfgfile=None, default=None):
    '''read `cfgfile` file and optionally create one if it does not exist

    Args:
        cfgfile(str): path to configuration file to be read
        default(str): path to default configuration file that should be 
            used if cfgfile does not exist -- this is useful for creating
            config files on first run
    
    Returns:
        obj:configparser.ConfigParser
    '''
    if not cfgfile:
        return {}
    
    cfgfile = Path(cfgfile).expanduser()
    default = Path(default).expanduser() if default else None
    
    # check if specified file exists, otherwise copy the default (if provided)
    if not cfgfile.exists() and default:
        logging.debug(f'copying {default} to {cfgfile}')
        with cfgfile.open(mode='xb') as fid:
            fid.write(default.read_bytes())
    elif not default:
        raise FileNotFoundError(f'could not open {cfgfile}')
    
    config = configparser.ConfigParser()
    logging.info(f'reading config file {cfgfile}')
    config.read(cfgfile)
    
    return config

In [ ]:
import sys

In [ ]:
sys.argv.append('-c')
sys.argv.append('~/slimpi.ini')

In [ ]:
sys.argv


In [ ]:
args = arg_parse()

In [ ]:
print(args.q)


In [ ]:
cfg = read_config(args.c, required)

In [ ]:
cfg.sections()

In [ ]:
len(cfg.sections())

In [ ]:
if len(cfg.sections()) < 1:
    print('no sections')

In [58]:
def main():
    # Configuration Variables
    
    # name
    appShortName = 'slimpi'
    name = 'com.txoof.'
    appLongName = name+appShortName        
    
    # Default configuration file if none is specified
    builtin_cfg = Path('./slimpi.cfg')
    default_cfg = Path(f'~/.config/{appLongName}/slimpi.cfg').expanduser()  
    
    args = arg_parse()
    
    if args.c:
        cfg_file = args.c
    else:
        logging.info(f'using default configuration file: {default_cfg}')        
        cfg_file = default_cfg
    
    # read configuration file
    configuration = read_config(cfgfile=cfg_file, default=default_cfg)
    # check if there are any 
    if len(configuration.sections()) < 1:
        raise EOFError(f'config file, {cfg_file}, contained no configuration')

In [59]:
main()

<ipython-input-5-b795292e2218>:arg_parse:19:INFO - discarding unknwon commandline arguments: ['-f', '/Users/aciuffo/Library/Jupyter/runtime/kernel-e416910d-f78b-424b-957d-b4422452ccc5.json']
<ipython-input-58-1d167caca2cf>:main:18:INFO - using default configuration file: /Users/aciuffo/.config/com.txoof.slimpi/slimpi.cfg
<ipython-input-42-8fd333b236b2>:read_config:21:DEBUG - copying /Users/aciuffo/.config/com.txoof.slimpi/slimpi.cfg to /Users/aciuffo/.config/com.txoof.slimpi/slimpi.cfg


FileNotFoundError: [Errno 2] No such file or directory: '/Users/aciuffo/.config/com.txoof.slimpi/slimpi.cfg'

In [12]:
a = 'com.txoof.slimpi'

In [56]:
p = Path(f'~/.config/{a}/slimpi.cfg').expanduser()

In [57]:
p

PosixPath('/Users/aciuffo/.config/com.txoof.slimpi/slimpi.cfg')

In [51]:
dir(p)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__func__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [22]:
if p.exists():
    print('ok')
else:
    print('make stuff')

make stuff
